dataset: https://www.kaggle.com/ashaheedq/video-games-sales-2019?select=vgsales-12-4-2019.csv

# Dependencies

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

import os
import numpy as np
import tensorflow as tf
import pandas as pd
pd.set_option('display.max_columns', None)
from tensorflow import keras 
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from IPython.display import HTML

# Loading and Processing Data

In [2]:
#read in train csv
df=pd.read_csv('resources/vgsales-12-4-2019.csv')
vgames = df.copy()
vgames.head()

,Rank,Name,basename,Genre,ESRB_Rating,Platform,Publisher,Developer,VGChartz_Score,Critic_Score,User_Score,Total_Shipped,Global_Sales,NA_Sales,PAL_Sales,JP_Sales,Other_Sales,Year,Last_Update,url,status,Vgchartzscore,img_url
0,1,Wii Sports,wii-sports,Sports,E,Wii,Nintendo,Nintendo EAD,NaN,7.7,NaN,82.86,NaN,NaN,NaN,NaN,NaN,2006.0,NaN,http://www.vgchartz.com/game/2667/wii-sports/?...,1,NaN,/games/boxart/full_2258645AmericaFrontccc.jpg
1,2,Super Mario Bros.,super-mario-bros,Platform,NaN,NES,Nintendo,Nintendo EAD,NaN,10.0,NaN,40.24,NaN,NaN,NaN,NaN,NaN,1985.0,NaN,http://www.vgchartz.com/game/6455/super-mario-...,1,NaN,/games/boxart/8972270ccc.jpg
2,3,Mario Kart Wii,mario-kart-wii,Racing,E,Wii,Nintendo,Nintendo EAD,NaN,8.2,9.1,37.14,NaN,NaN,NaN,NaN,NaN,2008.0,11th Apr 18,http://www.vgchartz.com/game/6968/mario-kart-w...,1,8.7,/games/boxart/full_8932480AmericaFrontccc.jpg
3,4,PlayerUnknown's Battlegrounds,playerunknowns-battlegrounds,Shooter,NaN,PC,PUBG Corporation,PUBG Corporation,NaN,NaN,NaN,36.60,NaN,NaN,NaN,NaN,NaN,2017.0,13th Nov 18,http://www.vgchartz.com/game/215988/playerunkn...,1,NaN,/games/boxart/full_8052843AmericaFrontccc.jpg
4,5,Wii Sports Resort,wii-sports-resort,Sports,E,Wii,Nintendo,Nintendo EAD,NaN,8.0,8.8,33.09,NaN,NaN,NaN,NaN,NaN,2009.0,NaN,http://www.vgchartz.com/game/24656/wii-sports-...,1,8.8,/games/boxart/full_7295041AmericaFrontccc.jpg


In [3]:
print(vgames.columns)

Index(['Rank', 'Name', 'basename', 'Genre', 'ESRB_Rating', 'Platform',
       'Publisher', 'Developer', 'VGChartz_Score', 'Critic_Score',
       'User_Score', 'Total_Shipped', 'Global_Sales', 'NA_Sales', 'PAL_Sales',
       'JP_Sales', 'Other_Sales', 'Year', 'Last_Update', 'url', 'status',
       'Vgchartzscore', 'img_url'],
      dtype='object')


In [4]:
vgames.drop(['Total_Shipped', 'Last_Update', 'url', 'status', 'Vgchartzscore', 'img_url'], axis=1, inplace=True)
vgames.head()

,Rank,Name,basename,Genre,ESRB_Rating,Platform,Publisher,Developer,VGChartz_Score,Critic_Score,User_Score,Global_Sales,NA_Sales,PAL_Sales,JP_Sales,Other_Sales,Year
0,1,Wii Sports,wii-sports,Sports,E,Wii,Nintendo,Nintendo EAD,NaN,7.7,NaN,NaN,NaN,NaN,NaN,NaN,2006.0
1,2,Super Mario Bros.,super-mario-bros,Platform,NaN,NES,Nintendo,Nintendo EAD,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,1985.0
2,3,Mario Kart Wii,mario-kart-wii,Racing,E,Wii,Nintendo,Nintendo EAD,NaN,8.2,9.1,NaN,NaN,NaN,NaN,NaN,2008.0
3,4,PlayerUnknown's Battlegrounds,playerunknowns-battlegrounds,Shooter,NaN,PC,PUBG Corporation,PUBG Corporation,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017.0
4,5,Wii Sports Resort,wii-sports-resort,Sports,E,Wii,Nintendo,Nintendo EAD,NaN,8.0,8.8,NaN,NaN,NaN,NaN,NaN,2009.0


In [5]:
vgames.drop(['Developer', 'VGChartz_Score', 'Name','User_Score'], axis=1, inplace=True)
vgames.head()

,Rank,basename,Genre,ESRB_Rating,Platform,Publisher,Critic_Score,Global_Sales,NA_Sales,PAL_Sales,JP_Sales,Other_Sales,Year
0,1,wii-sports,Sports,E,Wii,Nintendo,7.7,NaN,NaN,NaN,NaN,NaN,2006.0
1,2,super-mario-bros,Platform,NaN,NES,Nintendo,10.0,NaN,NaN,NaN,NaN,NaN,1985.0
2,3,mario-kart-wii,Racing,E,Wii,Nintendo,8.2,NaN,NaN,NaN,NaN,NaN,2008.0
3,4,playerunknowns-battlegrounds,Shooter,NaN,PC,PUBG Corporation,NaN,NaN,NaN,NaN,NaN,NaN,2017.0
4,5,wii-sports-resort,Sports,E,Wii,Nintendo,8.0,NaN,NaN,NaN,NaN,NaN,2009.0


In [6]:
vgames.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55792 entries, 0 to 55791
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Rank          55792 non-null  int64  
 1   basename      55792 non-null  object 
 2   Genre         55792 non-null  object 
 3   ESRB_Rating   23623 non-null  object 
 4   Platform      55792 non-null  object 
 5   Publisher     55792 non-null  object 
 6   Critic_Score  6536 non-null   float64
 7   Global_Sales  19415 non-null  float64
 8   NA_Sales      12964 non-null  float64
 9   PAL_Sales     13189 non-null  float64
 10  JP_Sales      7043 non-null   float64
 11  Other_Sales   15522 non-null  float64
 12  Year          54813 non-null  float64
dtypes: float64(7), int64(1), object(5)
memory usage: 5.5+ MB


In [7]:
vgames.isnull().sum()

Rank                0
basename            0
Genre               0
ESRB_Rating     32169
Platform            0
Publisher           0
Critic_Score    49256
Global_Sales    36377
NA_Sales        42828
PAL_Sales       42603
JP_Sales        48749
Other_Sales     40270
Year              979
dtype: int64

In [8]:
vgames = vgames.dropna()
vgames.head()

,Rank,basename,Genre,ESRB_Rating,Platform,Publisher,Critic_Score,Global_Sales,NA_Sales,PAL_Sales,JP_Sales,Other_Sales,Year
19,20,grand-theft-auto-v,Action,M,PS3,Rockstar Games,9.4,20.32,6.37,9.85,0.99,3.12,2013.0
20,21,grand-theft-auto-v,Action,M,PS4,Rockstar Games,9.7,19.39,6.06,9.71,0.60,3.02,2014.0
30,31,grand-theft-auto-vice-city,Action,M,PS2,Rockstar Games,9.6,16.15,8.41,5.49,0.47,1.78,2002.0
40,41,call-of-duty-modern-warfare-3,Shooter,M,X360,Activision,8.7,14.82,9.07,4.29,0.13,1.33,2011.0
41,42,call-of-duty-black-ops,Shooter,M,X360,Activision,8.8,14.74,9.76,3.73,0.11,1.14,2010.0


In [9]:
vgames.Genre.value_counts().head(50)

Role-Playing        226
Action              217
Shooter             200
Sports              118
Racing               97
Fighting             94
Platform             84
Adventure            64
Simulation           48
Misc                 42
Strategy             34
Puzzle               34
Action-Adventure     33
Party                 3
Music                 3
Sandbox               1
Board Game            1
MMO                   1
Name: Genre, dtype: int64

In [10]:
vgames = vgames.rename(columns={'basename': 'game'})
vgames.head()

,Rank,game,Genre,ESRB_Rating,Platform,Publisher,Critic_Score,Global_Sales,NA_Sales,PAL_Sales,JP_Sales,Other_Sales,Year
19,20,grand-theft-auto-v,Action,M,PS3,Rockstar Games,9.4,20.32,6.37,9.85,0.99,3.12,2013.0
20,21,grand-theft-auto-v,Action,M,PS4,Rockstar Games,9.7,19.39,6.06,9.71,0.60,3.02,2014.0
30,31,grand-theft-auto-vice-city,Action,M,PS2,Rockstar Games,9.6,16.15,8.41,5.49,0.47,1.78,2002.0
40,41,call-of-duty-modern-warfare-3,Shooter,M,X360,Activision,8.7,14.82,9.07,4.29,0.13,1.33,2011.0
41,42,call-of-duty-black-ops,Shooter,M,X360,Activision,8.8,14.74,9.76,3.73,0.11,1.14,2010.0


In [11]:
vgames.drop(['Rank'], axis=1, inplace=True)
vgames.to_csv('resources/vgames_clean.csv')

In [13]:
table = vgames.to_html()
print(table)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>game</th>
      <th>Genre</th>
      <th>ESRB_Rating</th>
      <th>Platform</th>
      <th>Publisher</th>
      <th>Critic_Score</th>
      <th>Global_Sales</th>
      <th>NA_Sales</th>
      <th>PAL_Sales</th>
      <th>JP_Sales</th>
      <th>Other_Sales</th>
      <th>Year</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>19</th>
      <td>grand-theft-auto-v</td>
      <td>Action</td>
      <td>M</td>
      <td>PS3</td>
      <td>Rockstar Games</td>
      <td>9.4</td>
      <td>20.32</td>
      <td>6.37</td>
      <td>9.85</td>
      <td>0.99</td>
      <td>3.12</td>
      <td>2013.0</td>
    </tr>
    <tr>
      <th>20</th>
      <td>grand-theft-auto-v</td>
      <td>Action</td>
      <td>M</td>
      <td>PS4</td>
      <td>Rockstar Games</td>
      <td>9.7</td>
      <td>19.39</td>
      <td>6.06</td>
      <td>9.71</td>
      <td>0.60</td>
      <td>3.02</td>
      <